# 支持向量机模型，直接使用sklearn库

## 1 读取图像数据

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch
import warnings
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time
import random
from sklearn.svm import SVC
import os
warnings.filterwarnings("ignore")
def set_random_seed(seed_value):
    """设置随机种子"""
    np.random.seed(seed_value)  # NumPy
    random.seed(seed_value)  # Python
    torch.manual_seed(seed_value)  # PyTorch CPU
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)  # PyTorch GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
#设置随机种子
set_random_seed(42)
#设置设备
device='cuda' if torch.cuda.is_available() else 'cpu'

#设置数据集超参数
batch_size=32
img_size=64

#定义图像变换
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),     #统一图像尺寸
    transforms.ToTensor(),                       #将图像转换为张量，且将像素值归一化到[0,1]，不需要手动除255
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )                                            #设置图像归一化参数，此处直接使用ImageNet的均值和标准差
])

#加载训练数据集
train_dataset = datasets.ImageFolder(root="data/train",transform=transform)
#加载验证数据集
val_dataset = datasets.ImageFolder(root="data/val",transform=transform)
#加载测试数据集
test_dataset = datasets.ImageFolder(root="data/test",transform=transform)

#创建训练数据加载器
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
#创建验证数据加载器
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
#创建测试数据加载器
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

#查看训练数据集的类别
print(train_dataset.classes)
print(train_dataset.class_to_idx)
#查看训练数据加载器中的一个批次数据
for images, labels in train_loader:
    print(images.shape)
    print(labels)
    break

#sklearn的支持向量机模型不支持tensor输入，需要对数据进行转换,从tensor转换为Numpy,且需要将图像展平
def extract_features(dataloader):
    features = []
    labels = []

    for imgs, targets in dataloader:
        # imgs: [B, C, H, W]
        imgs = imgs.reshape(imgs.size(0), -1)  # 展平为（B，C*H*W）
        features.append(imgs.numpy())
        labels.append(targets.numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

X_train, y_train = extract_features(train_loader)
X_val, y_val     = extract_features(val_loader)
X_test, y_test   = extract_features(test_loader)

print("训练特征维度:", X_train.shape)


['Bird-drop', 'Clean', 'Dusty', 'Electrical-damage', 'Physical-Damage', 'Snow-Covered']
{'Bird-drop': 0, 'Clean': 1, 'Dusty': 2, 'Electrical-damage': 3, 'Physical-Damage': 4, 'Snow-Covered': 5}
torch.Size([32, 3, 64, 64])
tensor([3, 4, 0, 4, 5, 4, 0, 0, 1, 5, 1, 3, 2, 3, 2, 0, 4, 0, 0, 2, 2, 0, 1, 2,
        0, 3, 5, 3, 0, 2, 1, 0])
训练特征维度: (929, 12288)


## 2 构建支持向量机模型

In [2]:
svc=SVC(kernel="rbf",gamma="scale",random_state=42)  #选择高斯核函数

## 3 训练模型

In [3]:
svc.fit(X_train,y_train)


,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


## 4 验证集评估

In [4]:
y_val_pred = svc.predict(X_val)

print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nValidation Classification Report:")
print(classification_report(
    y_val,
    y_val_pred,
    target_names=train_dataset.classes,
    digits=4
))

Validation Accuracy: 0.850909090909091

Validation Classification Report:
                   precision    recall  f1-score   support

        Bird-drop     0.8241    0.8558    0.8396       104
            Clean     0.7876    0.8725    0.8279       102
            Dusty     0.8043    0.7629    0.7831        97
Electrical-damage     0.8421    0.8312    0.8366        77
  Physical-Damage     0.9692    0.8077    0.8811        78
     Snow-Covered     0.9271    0.9674    0.9468        92

         accuracy                         0.8509       550
        macro avg     0.8591    0.8496    0.8525       550
     weighted avg     0.8542    0.8509    0.8509       550



## 5 测试集评估

In [5]:
y_test_pred = svc.predict(X_test)

print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nTest Classification Report:")
print(classification_report(
    y_test,
    y_test_pred,
    target_names=train_dataset.classes,
    digits=4
))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Test Accuracy: 0.8526315789473684

Test Classification Report:
                   precision    recall  f1-score   support

        Bird-drop     0.7895    0.8824    0.8333        17
            Clean     0.7778    0.7778    0.7778        18
            Dusty     0.8571    0.7500    0.8000        16
Electrical-damage     0.9167    0.8462    0.8800        13
  Physical-Damage     0.8667    0.8667    0.8667        15
     Snow-Covered     0.9412    1.0000    0.9697        16

         accuracy                         0.8526        95
        macro avg     0.8582    0.8538    0.8546        95
     weighted avg     0.8538    0.8526    0.8518        95


Confusion Matrix:
[[15  0  1  0  1  0]
 [ 1 14  1  1  1  0]
 [ 2  1 12  0  0  1]
 [ 0  2  0 11  0  0]
 [ 1  1  0  0 13  0]
 [ 0  0  0  0  0 16]]
